In [1]:
import numpy as np
from skimage.measure import marching_cubes
from scipy.ndimage import gaussian_filter
from os.path import join
from os import listdir
import json
import cv2

import meshio
import nibabel as nib

Define Atlas class

In [2]:

class Atlas:
    def __init__(self, annotationDirectory, calibrationAnnotation, save=False) -> None:
        # TODO: Get directory length
        self.calibration: float = None
        self.xOffset: int = None
        self.yOffset: int = None
        self.zOffset: int = None

        self.atlasDims: tuple = None
        self.affine = None

        self.organs = {}

        self.annotationDirectory = annotationDirectory

        ct = nib.load("./assets/images/sample/CT_TS_HEUHR_In111_free_M1039_0h_220721-selfcal.nii")
        ct = nib.load(join("assets", "images", "sagittal_mouse.nii"))

        annFiles = listdir(annotationDirectory)
        numSlices = 0
        for file in annFiles:
            annList = json.load(open(join(self.annotationDirectory, file)))
            numSlices += len(annList)

        self.calibrateDepth(calibrationAnnotation, numSlices)
        self.calibrateImgSize(ct)
        self.constructAtlasFromList(annFiles, numSlices)
        self.constructImgVoxels(save=save)

    def constructAtlasFromList(self, fileList, numSlices):
        if (self.calibration == None):
            print("Requires calibration")
            return
        # get number of slices
        for file in fileList:
            path = join(self.annotationDirectory, file)
            if (path == self.calibrationFile):
                print("Calibration file. Skipping...")
                continue
            print(f"Reading file {file}")
            annotationList = json.load(open(path))
            for annotation in annotationList:
                fname = annotation["documents"][0]["name"]
                index = int(fname.split('.')[0].replace("rat",""))
                try:
                    for entity in annotation["annotation"]["annotationGroups"][0]["annotationEntities"]:
                        name = entity["name"]
                        try:
                            organ = self.organs[name]
                        except(KeyError):
                            organ = Organ(name, 
                                            numSlices, 
                                            self.calibration, 
                                            self.atlasDims,
                                            self.affine)
                            self.organs[name] = organ
                        organ.appendOrganSlice(index, entity)
                except:
                    print(annotation)

    def calibrateDepth(self, calibrationAnnotation, numSlices):
        f = open(calibrationAnnotation)
        annotation = json.load(f)[0]
        body = annotation["annotation"]["annotationGroups"][0]["annotationEntities"][0]
        domain = []
        for point in body["annotationBlocks"][0]["annotations"][0]["segments"][0]:
            domain.append(point[1])
        minZ = min(domain)
        maxZ = max(domain)
        diff = maxZ - minZ
        self.calibration = 1.0 * float(diff) / float(numSlices)
        self.calibrationFile = calibrationAnnotation
        return self.calibration
    
    def calibrateImgSize(self, inputNifti):
        """
        Parameters:
            inputNifti: nibabel.nifti1.Nifti1Image
        """
        # TODO: use nifi image size to calibrate canvas size for voxelclouds
        shape = inputNifti.get_fdata().shape
        self.atlasDims = shape
        self.affine = inputNifti.affine
        print(self.affine, type(self.affine))
        pass

    def constructImgVoxels(self, save=False):
        for organName in self.organs:
            organ = None
            try:
                organ = self.organs[organName]
            except:
                continue
            if (organ != None):
                print(f"Constructing voxel map for {organName}")
                organ.constructVoxelMap(save)
                print("Done")



Define Organ class

In [15]:

class Organ:
    def __init__(self, name, numSlices, depth, dims, affine) -> None:
        self.name = name
        self.numSlices = numSlices
        self.slices = [[]] * numSlices
        self.scale = 1.0
        self.depth = depth * self.scale
        self.affine = affine
        # offset: [offset_x, offset_y, offset_z]
        self.offset = {
            "x": 250, # 124, # OFFSET_X - 36 * 6, # 424.2
            "y": 173, # 45, # OFFSET_Y - 80 * 6, # 33
            "z": 385 # 50 # 65
        }
        imgSliceDims = (numSlices, dims[1], dims[2])
        self.imageSlices: np.ndarray = np.zeros(imgSliceDims, dtype=np.uint8)
        self.voxelCloud: np.ndarray = np.zeros(dims, dtype=np.uint8)
    
    def appendOrganSlice(self, index, entity):
        for polygon in entity["annotationBlocks"][0]["annotations"]:
            polyPts = np.array(polygon["segments"][0].copy()).astype(int)
            for i, pt in enumerate(polyPts):
                polyPts[i] = [(self.scale * pt[1]) - self.offset['y'], (self.scale * pt[0]) - self.offset['x']]
            self.slices[index].append(polyPts)
            cv2.fillPoly(self.imageSlices[index], pts=[polyPts], color=(255, 255, 255))

    def constructVoxelMap(self, save=False):
        for z in range(self.voxelCloud.shape[0]):
            # i: voxel layer
            i = z - self.offset['z']
            index = float(i) / self.depth
            ind0 = int(np.floor(index))
            if (ind0 < 0):
                continue
            if (ind0 + 1 >= len(self.imageSlices)):
                break
            img0 = self.imageSlices[ind0]
            img1 = self.imageSlices[ind0 + 1]
            img0 = cv2.GaussianBlur(img0, (17, 17), cv2.BORDER_DEFAULT)
            img1 = cv2.GaussianBlur(img1, (17, 17), cv2.BORDER_DEFAULT)
            alpha = (float(i % int(np.round(self.depth)) ) ) / (self.depth)
            additiveImage = np.add(img0 *  (1.0 - alpha), img1 * alpha)
            self.voxelCloud[z] = additiveImage # [np.where(additiveImage > 196)] = 255
        
        self.customCalibration()
        self.generateMesh(save)

        if (save):
            img = nib.Nifti1Image(self.voxelCloud, self.affine)
            nib.save(img, f"./exports/{self.name}.nii")
            print(f"Saved {self.name} image at ./exports/{self.name}.nii")
            del self.imageSlices
            del self.slices
            del self.voxelCloud
    
    def customCalibration(self):
        self.voxelCloud = np.swapaxes(self.voxelCloud, 0, 1)
        self.voxelCloud = self.voxelCloud[::-1,::-1,::-1]

        # smooth between slices
        self.voxelCloud = gaussian_filter(self.voxelCloud, 15)
        self.voxelCloud[np.where(self.voxelCloud > 100)] = np.uint8(255)
        self.voxelCloud[np.where(self.voxelCloud <= 100)] = np.uint8(0)
        
    def generateMesh(self, save=False):
        threshold = 50
        step_size = 3
        print("Getting mesh")
        vertices, faces, _, _ = marching_cubes(self.voxelCloud, level=threshold, step_size=step_size)
        self.vertices = vertices
        self.faces = faces
        if (save):
            self.saveMesh()

    def getMesh(self):
        return self.vertices, self.faces

    def saveMesh(self):
        mesh = meshio.Mesh(self.vertices, {"triangle": self.faces})
        meshio.write(f"./exports/{self.name}.obj", mesh, file_format="obj")


Define file path for sagittal view images

In [13]:
SAGITTAL_FOLDER = "./assets/sagittal/"
DORSAL_FOLDER = "./assets/dorsal/"

Define some constants

In [5]:
dorsal_height = 1980-168
dorsal_width = 2480-1300
dorsal_ratio = float(dorsal_height)/float(dorsal_width)
sagittal_width = 2550-808
sagittal_height = 1617-483
sagittal_ratio = float(sagittal_width)/float(sagittal_height)

z_height = sagittal_height

Create list of images to interpolate between

In [6]:
num_dorsal = len(listdir(DORSAL_FOLDER))
num_sagittal = len(listdir(SAGITTAL_FOLDER))

dorsal_depth = 590 / num_dorsal
sagittal_depth = 590 / num_sagittal

dorsal_images = [[]] * num_dorsal
sagittal_images = [[]] * num_sagittal

def get_index(filename):
    return int(filename.replace("mouse","").replace(".png",""))

In [7]:
for image in listdir(SAGITTAL_FOLDER):
    if (image.split('.')[1] != "png"):
        continue
    index = get_index(image)
    im = cv2.imread(join(SAGITTAL_FOLDER, image))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    sagittal_im = im[483:1617, 808:2550]
    height = dorsal_width
    width = dorsal_height

    sagittal_im = cv2.resize(sagittal_im, (width, height), interpolation=cv2.INTER_AREA)
    sagittal_images[index] = sagittal_im

Create volume and fill with images

In [8]:
output_nifti = np.zeros((dorsal_width, dorsal_width, dorsal_height), dtype=np.uint8)

for z in range (272, 862):
    i = z - 272
    try:
        index = float(i) / sagittal_depth
        ind0 = int(np.floor(index))
        if (ind0 < 0):
            continue
        if (ind0 + 1 >= num_sagittal):
            break
        img0 = np.array(sagittal_images[ind0]).astype(float)
        img1 = np.array(sagittal_images[ind0 + 1]).astype(float)
        alpha = (float (i % int(np.round(sagittal_depth)))) / sagittal_depth
        additiveImg = np.add(img0 * (1.0 - alpha), img1 * alpha).astype(np.uint8)
        output_nifti[z] = additiveImg
    except:
        pass

Flip image along Z axis

In [10]:
output_nifti = output_nifti[::, ::, ::-1]

Save nifti image

In [11]:
from reg_utils import get_affine

affine = get_affine()

nifti_img = nib.Nifti1Image(output_nifti, affine=affine)
nib.save(nifti_img, "./atlas/reference/mouse_nii/atlas.nii")

Create organ atlas

In [16]:
atlas = Atlas(join("assets", "annotations"), join("assets", "annotations", "calibrate.json"), save=True)

[[ 0.16952001  0.          0.         21.58835983]
 [ 0.          0.16952001  0.         21.69745827]
 [ 0.          0.          0.168538   95.23029327]
 [ 0.          0.          0.          1.        ]] <class 'numpy.ndarray'>
Calibration file. Skipping...
Reading file annotations.json
Constructing voxel map for Stomach


MemoryError: Unable to allocate 56.2 GiB for an array with shape (2514131365, 3) and data type int64